In [1]:
import pandas as pd
ID_file = pd.read_csv("../uniprot-to-exon/transcript_ensembl.tab", sep = "\t")

In [18]:
import requests, sys
 
server = "https://rest.ensembl.org"

ext = "/sequence/id/"+ID_file.iloc[0,1]+"?object_type=transcript;type=cds"
r = requests.get(server+ext, headers={ "Content-Type" : "text/x-fasta"})
if not r.ok:
  r.raise_for_status()
print(r.text)

ext2 = "/sequence/id/"+ID_file.iloc[0,1]+"?object_type=transcript;type=genomic"
r2 = requests.get(server+ext2, headers={ "Content-Type" : "text/x-fasta"})
if not r2.ok:
  r2.raise_for_status()
print(r2.text)

>ENSOGAT00000002463.2
GTGGCTGGGAACCCAGCCCAGCCCTCTCCGGGGAGGGAGAGCCTGGCGCGCAGTAAGGCG
GCGCCTGGCGCCCCCTCGGGTCTGAGCTGTGCACAGCAAGGGTGCCGGGGGCGGCGCCCG
GCCCTGCCCCTGGGGGGCCGTTCTTGGAGGCCAGAGGCGGGGCAGGGCGTTGAGACCCCT
GAGCAAGAGAGCGAAGTCCTCCAGCCATCAGCCCCGCCTGCCGCGGCTGCAGGCTTTGTG
CGTCCTAAGCCAGGCAGTGACGATCCTGGGCCGTCCGAAGCGAGAAGCGCAGAGGTCCAG
GGGCAGTTTCCCCCTCCGGGCCCTAAGATCTCCTCTGGGTCAAGCCTGGAAACCCCGCAG
CTGAGTTTTCTTCTGTGTCCCGAAGATCTCCAACCAGACCCAGAGGAGCTGCCCATGGAG
ACGCCCATCGAGCGCGAAATCCGCCGCAGCTGCGAACGTGAAGAAAGCCTGCGGCGGAGC
CGGGGCCTGAGCCCGGGCCGCGCGGGCCGCGAATTCGTGGAGTTGCGTGTGCGGCCAGTG
CTCAGTCTGCCCGGCCCTGGCCCCGCGCCCACGCGCGCCTTGGAGCGCGCGCGCGCGGGT
GCGCAGATGCAGCGAGACATCGAGCGGGAGGCCCACCGGCAGGCGGCGCTAATGCGCCCC
GCGACCCCAGTCCCGCGCTCCGGGTCCCCGCCGCCGCTGGGTGAACTCAAGCGCTTCTTC
GAGGCCGCGGCGGAGAGCGGTTCCTCCTCGGTGGCGGCGGAAGGCGGCGTGAGCCCGCAG
TGGCGGCCAGAACCCGGAGGCCGTCCGCGCTCGGTAGTGCTGGCTCGCGCCCCGGCACCT
ATCGCTCCGTCGCTGCTAGAGCAGGAGGTGCGCGAGGTGCGCGAGCGCGAGCGGGAGCTG
CAGCGCCAGCGGCGCAGCGTCTACGGTACCGTAGACTTCAAGGACCCGGCGCCGAGCCTC
AC

In [117]:
import requests, sys
import json

server = "https://rest.ensembl.org"
ext = "/sequence/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

data = {"ids": ID_file.iloc[:1,1].tolist(), "type":"genomic"}
data = json.dumps(data)
r = requests.post(server+ext, headers=headers, data=data)
if not r.ok:
    print(r.text)
    r.raise_for_status()
    sys.exit()
    
decoded = r.json()

f = open("../uniprot-to-exon/genomics.fa", "w")
j = 0
for i in decoded:
    f.write(">"+i['query']+" "+str(ID_file.iloc[j,0]) + " GENOMIC\n")
    f.write(i['seq']+"\n")
    j = j+1
f.close

<function TextIOWrapper.close()>

In [3]:
import grequests
import json

url = "https://rest.ensembl.org/sequence/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

params = []
for i in range(0, 9960, 50):
    try:
        params.append({"ids": ID_file.iloc[i:i+50,1].tolist(), "type":"cds"})
    except:
        params.append({"ids": ID_file.iloc[i:,1].tolist(), "type":"cds"})

        
rs = [grequests.post(url, headers=headers, data=json.dumps(i)) for i in params]
all_response = grequests.map(rs, size=50)

C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
f = open("../uniprot-to-exon/cds_new.fa", "w")
indice = 0
for response in all_response:
    j = 0
    try:
        for entry in response.json():
            f.write(">"+entry['query']+" "+str(ID_file.iloc[indice*50+j,0]) + " CDS\n")
            f.write(entry['seq']+"\n")
            j = j+1
    except:
        pass
    indice = indice + 1

f.close()

C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
import grequests
import requests, sys
url = "https://rest.ensembl.org/lookup/id/"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

params = []
for i in range(0, 9960, 1000):
    try:
        params.append({"ids": ID_file.iloc[i:i+1000,1].tolist(), "expand":"1"})
    except:
        params.append({"ids": ID_file.iloc[i:,1].tolist(), "expand":"1"})

rs = [grequests.post(url, headers=headers, data=json.dumps(i)) for i in params]
all_response = grequests.map(rs)

C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [58]:
rs = [grequests.post(url, headers=headers, data=json.dumps(i)) for i in params]
all_response_V2 = grequests.map(rs)

In [67]:
f = open("../uniprot-to-exon/lookup_comma.json", "a")
for response in all_response_V2:
    try:
        f.write(json.dumps(response.json())+",")
    except:
        pass
f.close()

In [2]:
import json
import pandas
data_dict = []
with open('C:/Users/Glados/Documents/stage-thompson/uniprot-to-exon/lookup_newline.json') as json_data:
    for i in json_data:
        data_dict.append(json.loads(i))

def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)    
    return z

MyJsonFull = data_dict[0]
for i in data_dict[1:]:
    MyJsonFull = merge_two_dicts(MyJsonFull, i)

In [3]:
ABSOLUTE_POST = MyJsonFull['ENSNLET00000002590']["start"]
exon_list = MyJsonFull['ENSNLET00000002590']["Exon"]

counter = 1
for i in exon_list:
    print("Exon "+str(counter))
    print("Start corrected: ",  abs(i['start']-ABSOLUTE_POST))
    print("Stop corrected: ",  abs(i['end']-ABSOLUTE_POST))
    counter+=1


Exon 1
Start corrected:  0
Stop corrected:  14
Exon 2
Start corrected:  239
Stop corrected:  315
Exon 3
Start corrected:  1145
Stop corrected:  2105
Exon 4
Start corrected:  6823
Stop corrected:  7043
Exon 5
Start corrected:  7298
Stop corrected:  7491
Exon 6
Start corrected:  9341
Stop corrected:  9441


In [4]:
exon_list[0]

{'object_type': 'Exon',
 'strand': 1,
 'version': 1,
 'db_type': 'core',
 'seq_region_name': 'GL397444.1',
 'assembly_name': 'Nleu_3.0',
 'start': 295204,
 'species': 'nomascus_leucogenys',
 'id': 'ENSNLEE00000294789',
 'end': 295218}